In [4]:
# %load model.py
import random

from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector


def compute_gini(model):
    agent_wealths = [agent.wealth for agent in model.schedule.agents]
    x = sorted(agent_wealths)
    N = model.num_agents
    B = sum(xi * (N - i) for i, xi in enumerate(x)) / (N * sum(x))
    return (1 + (1 / N) - 2 * B)


class MoneyModel(Model):
    """A simple model of an economy where agents exchange currency at random.

    All the agents begin with one unit of currency, and each time step can give
    a unit of currency to another agent. Note how, over time, this produces a
    highly skewed distribution of wealth.
    """

    def __init__(self, N, width, height):
        self.num_agents = N
        self.grid = MultiGrid(height, width, True)
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            model_reporters={"Gini": compute_gini},
            agent_reporters={"Wealth": "wealth"}
        )
        # Create agents
        for i in range(self.num_agents):
            a = MoneyAgent(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)

    def run_model(self, n):
        for i in range(n):
            self.step()


class MoneyAgent(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 1

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos, moore=True, include_center=False
        )
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        if len(cellmates) > 1:
            other = random.choice(cellmates)
            other.wealth += 1
            self.wealth -= 1

    def step(self):
        self.move()
        if self.wealth > 0:
            self.give_money()


In [5]:
%load_ext autoreload


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from mesa.visualization.JupyterVisualizer import (
    JupyterVisualizer,
)
from mesa.visualization.UserParam import UserSettableParameter

from mesa.visualization.modules import (
    CanvasGrid, ChartModule,
)

import ipywidgets as widgets
from IPython.display import (
    display,
    Javascript,
    HTML,
)



def agent_portrayal(agent):
    portrayal = {"Shape": "circle",
                 "Filled": "true",
                 "r": 0.5}

    if agent.wealth > 0:
        portrayal["Color"] = "red"
        portrayal["Layer"] = 0
    else:
        portrayal["Color"] = "grey"
        portrayal["Layer"] = 1
        portrayal["r"] = 0.2
    return portrayal


grid = CanvasGrid(agent_portrayal, 10, 10, 500, 500)


chart = ChartModule(
    [{"Label": "Gini", "Color": "Black"}],
    data_collector_name='datacollector')
# From ChartVisualization.py: 
# chart.package_includes = ["Chart.min.js", "ChartModule.js"]
# Revove Chart.min.js
chart.package_includes = ["ChartModule.js"]
# And load it directly. Path must reference a valid installation of Chart.js.
custom_js = """ 
    require.config({{
        paths: {{
            chartjs: 'https://cdnjs.cloudflare.com/ajax/libs/Chart.js/2.7.1/Chart.min'
        }}
    }}); 

    require(['chartjs'], function (chart) {{
        // Important to add it to the namespace, otherwise Chart.js doesn't load
        window.Chart = chart; 
        {0}
    }});
"""
# display(Javascript(
# """ 
# require.config({
#     paths: {
#         chartjs: '/files/mesa/visualization/templates/js/Chart.min'
#     }
# }); 
        
# require(['chartjs'], function (chart) { 
#     // Important to add it to the namespace, otherwise Chart.js doesn't load
#     window.Chart = chart; 

# });"""))


model_params = {
    "N": UserSettableParameter('slider', "Number of agents", 100, 2, 200, 1,
                               description="Choose how many agents to include in the model"),
    "width": 10,
    "height": 10
}
v = JupyterVisualizer(MoneyModel, [grid,chart], 'Money Model', model_params)
v.display(custom_js)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Button(description='Play', style=ButtonStyle())

Button(description='Step The Model', style=ButtonStyle())

Button(description='Reset The Model', style=ButtonStyle())

IntSlider(value=10, continuous_update=False, description='Frames per second:', max=20, min=1)

<IPython.core.display.Javascript object>